## 导入工具包

In [1]:
import os
import json
import shutil
import numpy as np
from tqdm import tqdm

### 删除多余文件

## 数据集类别信息

In [2]:
Dataset_root = 'hand_enhance'

In [3]:
# 框的类别
bbox_class = {
    'hand':0  
}

In [4]:
# 关键点的类别
keypoint_class = ['zhongchong', 'shangyang', 'zhongkui','shixuan','guanchong','shaochong','shaoze','xiaogukong','yemen','hegu']

In [5]:
os.chdir(Dataset_root)

In [6]:
os.mkdir('labels')
os.mkdir('labels/train')
os.mkdir('labels/val')

## 函数-处理单个labelme标注json文件

In [7]:
def process_single_json(labelme_path, save_folder='../../labels/train'):
    
    with open(labelme_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)

    img_width = labelme['imageWidth']   # 图像宽度
    img_height = labelme['imageHeight'] # 图像高度

    # 生成 YOLO 格式的 txt 文件
    suffix = labelme_path.split('.')[-2]
    yolo_txt_path = suffix + '.txt'

    with open(yolo_txt_path, 'w', encoding='utf-8') as f:

        for each_ann in labelme['shapes']: # 遍历每个标注

            if each_ann['shape_type'] == 'rectangle': # 每个框，在 txt 里写一行

                yolo_str = ''

                ## 框的信息
                # 框的类别 ID
                bbox_class_id = bbox_class[each_ann['label']]
                yolo_str += '{} '.format(bbox_class_id)
                # 左上角和右下角的 XY 像素坐标
                bbox_top_left_x = int(min(each_ann['points'][0][0], each_ann['points'][1][0]))
                bbox_bottom_right_x = int(max(each_ann['points'][0][0], each_ann['points'][1][0]))
                bbox_top_left_y = int(min(each_ann['points'][0][1], each_ann['points'][1][1]))
                bbox_bottom_right_y = int(max(each_ann['points'][0][1], each_ann['points'][1][1]))
                # 框中心点的 XY 像素坐标
                bbox_center_x = int((bbox_top_left_x + bbox_bottom_right_x) / 2)
                bbox_center_y = int((bbox_top_left_y + bbox_bottom_right_y) / 2)
                # 框宽度
                bbox_width = bbox_bottom_right_x - bbox_top_left_x
                # 框高度
                bbox_height = bbox_bottom_right_y - bbox_top_left_y
                # 框中心点归一化坐标
                bbox_center_x_norm = bbox_center_x / img_width
                bbox_center_y_norm = bbox_center_y / img_height
                # 框归一化宽度
                bbox_width_norm = bbox_width / img_width
                # 框归一化高度
                bbox_height_norm = bbox_height / img_height

                yolo_str += '{:.5f} {:.5f} {:.5f} {:.5f} '.format(bbox_center_x_norm, bbox_center_y_norm, bbox_width_norm, bbox_height_norm)

                ## 找到该框中所有关键点，存在字典 bbox_keypoints_dict 中
                bbox_keypoints_dict = {}
                for each_ann in labelme['shapes']: # 遍历所有标注
                    if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                        # 关键点XY坐标、类别
                        x = int(each_ann['points'][0][0])
                        y = int(each_ann['points'][0][1])
                        label = each_ann['label']
                        if (x>bbox_top_left_x) & (x<bbox_bottom_right_x) & (y<bbox_bottom_right_y) & (y>bbox_top_left_y): # 筛选出在该个体框中的关键点
                            bbox_keypoints_dict[label] = [x, y]

                ## 把关键点按顺序排好
                for each_class in keypoint_class: # 遍历每一类关键点
                    if each_class in bbox_keypoints_dict:
                        keypoint_x_norm = bbox_keypoints_dict[each_class][0] / img_width
                        keypoint_y_norm = bbox_keypoints_dict[each_class][1] / img_height
                        yolo_str += '{:.5f} {:.5f} {} '.format(keypoint_x_norm, keypoint_y_norm, 2) # 2-可见不遮挡 1-遮挡 0-没有点
                    else: # 不存在的点，一律为0
                        yolo_str += '0 0 0 '
                # 写入 txt 文件中
                f.write(yolo_str + '\n')
                
    shutil.move(yolo_txt_path, save_folder)
    print('{} --> {} 转换完成'.format(labelme_path, yolo_txt_path))    

## 转换训练集标注文件至`labels/train`目录

In [8]:
os.chdir('labelme_jsons/train')

In [9]:
save_folder = '../../labels/train'
for labelme_path in os.listdir():
    try:
        process_single_json(labelme_path, save_folder=save_folder)
    except:
        print('******有误******', labelme_path)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

0000.json --> 0000.txt 转换完成
0001.json --> 0001.txt 转换完成
0002.json --> 0002.txt 转换完成
0003.json --> 0003.txt 转换完成
0004.json --> 0004.txt 转换完成
0005.json --> 0005.txt 转换完成
0006.json --> 0006.txt 转换完成
0007.json --> 0007.txt 转换完成
0010.json --> 0010.txt 转换完成
0011.json --> 0011.txt 转换完成
0012.json --> 0012.txt 转换完成
0013.json --> 0013.txt 转换完成
0014.json --> 0014.txt 转换完成
0017.json --> 0017.txt 转换完成
0018.json --> 0018.txt 转换完成
0019.json --> 0019.txt 转换完成
0020.json --> 0020.txt 转换完成
0022.json --> 0022.txt 转换完成
0023.json --> 0023.txt 转换完成
0026.json --> 0026.txt 转换完成
0027.json --> 0027.txt 转换完成
0029.json --> 0029.txt 转换完成
0031.json --> 0031.txt 转换完成
0032.json --> 0032.txt 转换完成
0033.json --> 0033.txt 转换完成
0034.json --> 0034.txt 转换完成
0035.json --> 0035.txt 转换完成
0036.json --> 0036.txt 转换完成
0040.json --> 0040.txt 转换完成
0041.json --> 0041.txt 转换完成
0043.json --> 0043.txt 转换完成
0044.json --> 0044.txt 转换完成
0045.json --> 0045.txt 转换完成
0046.json --> 0046.txt 转换完成
0047.json --> 0047.txt 转换完成
0049.json --> 0049.t

In [10]:
os.chdir('../../')

## 转换测试集标注文件至`labels/val`目录

In [11]:
os.chdir('labelme_jsons/val')

In [12]:
save_folder = '../../labels/val'
for labelme_path in os.listdir():
    try:
        process_single_json(labelme_path, save_folder=save_folder)
    except:
        print('******有误******', labelme_path)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

0008.json --> 0008.txt 转换完成
0009.json --> 0009.txt 转换完成
0015.json --> 0015.txt 转换完成
0016.json --> 0016.txt 转换完成
0021.json --> 0021.txt 转换完成
0024.json --> 0024.txt 转换完成
0025.json --> 0025.txt 转换完成
0028.json --> 0028.txt 转换完成
0030.json --> 0030.txt 转换完成
0037.json --> 0037.txt 转换完成
0038.json --> 0038.txt 转换完成
0039.json --> 0039.txt 转换完成
0042.json --> 0042.txt 转换完成
0048.json --> 0048.txt 转换完成
0058.json --> 0058.txt 转换完成
0061.json --> 0061.txt 转换完成
0064.json --> 0064.txt 转换完成
0070.json --> 0070.txt 转换完成
0071.json --> 0071.txt 转换完成
0074.json --> 0074.txt 转换完成
0078.json --> 0078.txt 转换完成
0080.json --> 0080.txt 转换完成
0082.json --> 0082.txt 转换完成
0092.json --> 0092.txt 转换完成
0093.json --> 0093.txt 转换完成
0100.json --> 0100.txt 转换完成
0106.json --> 0106.txt 转换完成
0109.json --> 0109.txt 转换完成
0110.json --> 0110.txt 转换完成
0117.json --> 0117.txt 转换完成
0120.json --> 0120.txt 转换完成
0121.json --> 0121.txt 转换完成
0126.json --> 0126.txt 转换完成
0127.json --> 0127.txt 转换完成
0129.json --> 0129.txt 转换完成
0132.json --> 0132.t

In [13]:
os.chdir('../../')

## 删除labelme格式的标注文件

In [ ]:
!rm -rf labelme_jsons

In [15]:
os.chdir('../')